# PassportCard Insurance Claims Prediction

This notebook develops a machine learning system to predict future insurance claims for PassportCard policyholders.

In [ ]:
print("Welcome to the Insurance Claims Prediction notebook!")